In [1]:
import sqlite3

conn = sqlite3.connect("database.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS submissions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code TEXT,
    errors TEXT,
    style_issues TEXT,
    suggestions TEXT,
    submitted_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")

conn.commit()
print("Database and table created successfully")


Database and table created successfully


In [2]:
import ast

def analyze_code(code):
    errors = []
    style_issues = []
    suggestions = []

    # Step 1: Syntax check
    try:
        tree = ast.parse(code)
    except SyntaxError as e:
        errors.append(f"Syntax Error: {e}")
        return errors, style_issues, suggestions

    # Step 2: AST analysis
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                style_issues.append(f"Check if import '{alias.name}' is necessary")

        if isinstance(node, ast.FunctionDef):
            if len(node.body) > 15:
                style_issues.append(f"Function '{node.name}' is too long")

    # Step 3: Simple AI-like suggestions
    if "range(len" in code:
        suggestions.append(
            "Avoid using range(len()); iterate directly over elements."
        )

    if not errors:
        suggestions.append("Code looks good. Try adding comments for clarity.")

    return errors, style_issues, suggestions


In [3]:
student_code = """
import math

def calculate(nums):
    total = 0
    for i in range(len(nums)):
        total += nums[i]
    return total
"""


In [4]:
errors, style, suggestions = analyze_code(student_code)

cursor.execute("""
INSERT INTO submissions (code, errors, style_issues, suggestions)
VALUES (?, ?, ?, ?)
""", (
    student_code,
    ", ".join(errors),
    ", ".join(style),
    ", ".join(suggestions)
))

conn.commit()

print("---- Code Review Feedback ----")
print("Errors:", errors)
print("Style Issues:", style)
print("Suggestions:", suggestions)


---- Code Review Feedback ----
Errors: []
Style Issues: ["Check if import 'math' is necessary"]
Suggestions: ['Avoid using range(len()); iterate directly over elements.', 'Code looks good. Try adding comments for clarity.']


In [5]:
cursor.execute("SELECT id, submitted_at FROM submissions")
rows = cursor.fetchall()

print("Submission History:")
for row in rows:
    print(row)



Submission History:
(1, '2026-01-06 09:36:27')


In [6]:
conn.close()
print("Database connection closed")


Database connection closed
